<a href="https://colab.research.google.com/gist/adeshsankpal/3da697bd42e94ac250812f4dc010a5df/xml-reader-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import xml.etree.ElementTree as ETree
import pandas as pd
import html
#decoded_xml_snippet = html.unescape(xml_snippet)

In [ ]:
def workbook(root):
  rd=[]
  for i in root.iter('worksheet'):
      try:
          #print(i.attrib['name'])
          ds_list=[]
          data_list=[]
          attr=i.attrib
          #print("<>")
          for ds in i.iter('datasource'):
              dct1=ds.attrib
              ds_list.append(dct1)
          df_ds_map=pd.DataFrame(ds_list)
          for j in i.iter('datasource-dependencies'):
              dct=j.attrib
              #print(dct)
              #print("break/n")
              for cl in j.iter('column'):
                  col=cl.attrib
                  col['datasource']=dct['datasource']
                  col['worksheet']=i.attrib['name']
                  #print("this is new column")
                  if cl.find('calculation') is not None:
                      for cal in cl.iter('calculation'):
                          cal_list=cal.attrib
                          col['calculation']=cal_list['formula']
                  else:
                      col['calculation']=""


                  #for cal in cl.iter('calculation'):
                      #cal_list=cal.attrib
                  #col['calculation']=cal_list['formula']
                  #col['calculation']=''


                  if all(key in col for key in ("caption", "name","datasource")):
                      col['name']=col['name'].replace(']','')
                      col['name']=col['name'].replace('[','')
                      data_list.append(col)
                  elif all(key in col for key in ("datatype", "name","datasource")):
                      col['caption']=col['name']
                      col['name']=col['name'].replace(']','')
                      col['name']=col['name'].replace('[','')
                      col['caption']=col['caption'].replace(']','')
                      col['caption']=col['caption'].replace('[','')
                      data_list.append(col)
          df_data=pd.DataFrame(data_list)
          #print(df_data.head())

          df_ds_map.rename(columns = {'caption':'table_name'}, inplace = True)
          df_ds_map.rename(columns = {'name':'datasource'}, inplace = True)

          final_data=pd.merge(df_data,df_ds_map,on=["datasource"],how="left")
          final_data=final_data[["caption","name","table_name","worksheet","calculation"]]
          rd.append(final_data)
      except:
          t=0

  wb_data=pd.concat(rd)
  # Convert 'calculation' column to pandas.Series
  wb_data['calculation'] = pd.Series(wb_data['calculation'])

  # Decoding the 'calculation' column
  wb_data['calculation'] = wb_data['calculation'].apply(html.unescape)
  wb_data=replace_calc(wb_data)
  wb_data=wb_data.groupby(['table_name','worksheet','caption','name','calculation'],as_index=True).sum(1)
  return wb_data


In [ ]:
def replace_calc(wb_data):
  wb_data=wb_data.reset_index()
  iter_data=wb_data.groupby(['name', 'caption'],as_index=False).sum(1)
  for i in range(len(iter_data['name'])):
    x_name=iter_data['name'][i]
    x_caption=iter_data['caption'][i]
    for j in range(len(wb_data['calculation'])):
      cal=wb_data['calculation'][j]
      if "["+x_name+"]" in cal:
        wb_data['calculation'][j]=wb_data['calculation'][j].replace("["+x_name+"]","["+x_caption+"]")
  return wb_data



In [ ]:
xmldata=r"/content/drive/My Drive/Tableau Workbooks/Sample Superstore RFM Customer Segmentation.xml"
prstree = ETree.parse(xmldata)
root = prstree.getroot()

In [ ]:
df=workbook(root)

<ipython-input-4-78e948f115bb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_data['calculation'][j]=wb_data['calculation'][j].replace("["+x_name+"]","["+x_caption+"]")


In [ ]:
output_path=r"/content/drive/My Drive/Tableau Workbooks/tableau_documents.xlsx"
with open(output_path,"w") as file:
  file.close()
df.to_excel(output_path, index=True)

In [ ]:
df.head(50)

index
table_name         worksheet            caption                  name                                             calculation                                              
Superstore_storedf # Fixed Avg Discount 1                        Calculation_651051641304797190                   1                                                       0
                                        Customer ID              Customer ID                                                                                              1
                                        Discount                 Discount                                                                                                 2
                                        Fixed Avg Discount       Fixed Avg Sales (copy)_1637902910375907333       {FIXED [Customer ID]: AVG([Discount])}                  3
                   # Fixed Avg Profit   1                        Calculation_651051641304797190                   1                                                       0
                                        Customer ID              Customer ID                                                                                              2
                                        Fixed Avg Profit         Calculation_700872712705363981                   AVG({FIXED [Customer ID]: SUM([Profit])})               1
                                        Profit                   Profit                                                                                                   3
                   # Fixed Avg Sales    1                        Calculation_651051641304797190                   1                                                       0
                                        Customer ID              Customer ID                                                                                              1
                                        Fixed Avg Sales          Fixed Avg Profit (copy)_1637902910375854084      AVG({FIXED [Customer ID]: SUM([Sales])})                2
                                        Sales                    Sales                                                                                                    3
                   # Loyalty Disc       1                        Calculation_651051641304797190                   1                                                       1
                                        Customer ID              Customer ID                                                                                              2
                                        Discount                 Discount                                                                                                 3
                                        Loyalty                  Loyalty                                                                                                  4
                                        O Avg Discount           O Avg Profit (copy)_1637902910370365440          IF [Parameters].[Select Loyalty (All)] = "(All)...      5
                                        U Avg Discount           U Avg Profit (copy)_1637902910370484225          IF [Parameters].[Select Loyalty (All)] = "(All)...      6
                                        Yearly Avg Discount cust Yearly Avg Sales cust (copy)_1382323633674596355 AVG({FIXED [Loyalty],[Customer ID]: AVG([Discou...      7
                   # Loyalty Profit     1                        Calculation_651051641304797190                   1                                                       1
                                        Customer ID              Customer ID                                                                                              3
                                        Loyalty                  Loyalty                                                                                                  4
                                      